In [21]:
import sympy
from openLoop.utilities.ipynb.displays import *
from openLoop.utilities.ipynb.ipy_sympy import *
import scipy.linalg


import numpy.testing as np_test
import declarative

from test_SVD import SVD_gen_check, gen_rand_unitary
from openLoop.matrix import DAG_algorithm
from openLoop.matrix import SRE_matrix_algorithms
from openLoop.matrix import scisparse_algorithm

asavefig.org_subfolder = 'plots'

In [22]:
from functools import reduce
def SVD_compare_error(
    N = 10,
    length = 10,
    solvers = [
        scisparse_algorithm, 
        DAG_algorithm
    ],
):
    U = gen_rand_unitary(N = N, length = length)
    V = gen_rand_unitary(N = N, length = length)

    seq = dict()
    req = dict()
    edge_map = dict()
    S_diags = []
    for idx in range(N):
        s_diag = 10**(-5 + 10 * np.random.random(length))
        edge_map[idx, idx] = s_diag
        S_diags.append(s_diag)
        seq[idx] = set([idx])
        req[idx] = set([idx])
    S = seq, req, edge_map
    condition = reduce(np.maximum, S_diags) / reduce(np.minimum, S_diags)

    M = SRE_matrix_algorithms.matrix_mult_sre(
        SRE_matrix_algorithms.matrix_mult_sre(U, S), V
    )

    SRE_matrix_algorithms.check_sre(M)

    print("SPARSITY FRAC: ", SRE_matrix_algorithms.SRE_count_sparsity(M))

    inputs_set = set(range(N))
    outputs_set = set(range(N))

    def solve(solver):
        Mseq, Mreq, Medge_map = SRE_matrix_algorithms.copy_sre(M)
        print(solver)
        sbunch = solver.inverse_solve_inplace(
            seq = Mseq,
            req = Mreq,
            edge_map = Medge_map,
            inputs_set = inputs_set,
            outputs_set = outputs_set,
            verbose = True,
            negative = False,
        )

        Minv = sbunch.seq, sbunch.req, sbunch.edge_map
        SRE_matrix_algorithms.check_sre(M)
        SRE_matrix_algorithms.check_sre(Minv)

        Meye = SRE_matrix_algorithms.matrix_mult_sre(M, Minv)
        #pprint(Meye)
        em1_ll = dict()
        em0_ll = dict()
        for (k_t, k_f), edge in Meye[2].items():
            if (k_t == k_f):
                em1_ll[k_t, k_f] = -np.log10(np.maximum(abs(edge - 1), 1e-30))
            else:
                em0_ll[k_t, k_f] = -np.log10(np.maximum(abs(edge), 1e-30))
        
        all_e0 = []
        all_e1 = []
        
        for k, e in em0_ll.items():
            all_e0.append(e)
        for k, e in em1_ll.items():
            all_e1.append(e)
            
        if all_e0:
            all_e0 = np.concatenate(all_e0)
        else:
            all_e0 = []
            
        if all_e1:
            all_e1 = np.concatenate(all_e1)
        else:
            all_e1 = []
        return declarative.Bunch(
            solver = solver,
            Meye = Meye,
            em1_ll = em1_ll,
            em0_ll = em0_ll,
            all0 = all_e0,
            all1 = all_e1,
        )
    sdict = dict()
    for solver in solvers:
        sdict[solver] = solve(solver)
    sdict['condition'] = condition
    return sdict
    #pprint(em1_ll)
    #pprint(em0_ll)


In [23]:
r = SVD_compare_error(
    N = 100,
    length = 100,
)

SPARSITY FRAC:  {'density_lin': 3.13, 'Nedges': 313, 'Nnodes': 100, 'density_sq': 0.0313}
<module 'openLoop.matrix.scisparse_algorithm' from '/home/mcculler/local/home_sync/projects/openLoop/openLoop/matrix/scisparse_algorithm.py'>


/home/mcculler/.local/lib/python3.5/site-packages/scikits/umfpack/umfpack.py:718: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 2.69e+12)
  warnings.warn(msg, UmfpackWarning)
/home/mcculler/.local/lib/python3.5/site-packages/scikits/umfpack/umfpack.py:718: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 3.97e+12)
  warnings.warn(msg, UmfpackWarning)
/home/mcculler/.local/lib/python3.5/site-packages/scikits/umfpack/umfpack.py:718: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 3.63e+12)
  warnings.warn(msg, UmfpackWarning)
/home/mcculler/.local/lib/python3.5/site-packages/scikits/umfpack/umfpack.py:718: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.06e+15)
  warnings.warn(msg, UmfpackWarning)
/home/mcculler/.local/lib/python3.5/site-packages/scikits/umfpack/umfpack.py:718: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 6.76e+12)
  warnings.warn(msg, UmfpackWarning)
/home/mcculler/.loca

<module 'openLoop.matrix.DAG_algorithm' from '/home/mcculler/local/home_sync/projects/openLoop/openLoop/matrix/DAG_algorithm.py'>
SPARSITY  100 313 3.13
TRIVIAL STAGE, REMAINING 100
TRIVIAL STAGE, REMAINING 75
BADGUY STAGE, REMAINING 75
pqueue length:  75
MY NODE:  29
MIN_MAX:  29 29
SHAPE:  (100,)
32
REL LARGER:  32.0
Using COLUMN Operations
1 1 29
CVEC:  [0.999999999999999, 1.0, 0.93545436726507836, 0.43861627666492786]
[ True  True  True False] 1 3
pm_C:  3 4 True False
bignodes_c[cvec_self_idx] True
MUST USE HOUSEHOLDER 3x
(4,)
NFROM:  {26, 29, 46} 29
MY NODE:  60
MIN_MAX:  60 60
SHAPE:  (100,)
26
REL LARGER:  26.0
Using COLUMN Operations
8 8 60
CVEC:  [0.99998936078584388, 0.99876095650557495, 0.99999973098272799, 0.99994930414124528, 0.71873422290146416, 0.8981057614272181, 0.94901725775281132, 0.19890110832359589, 1.0, 0.94654986614789238]
[ True  True  True  True  True  True  True False  True  True] 8 9
pm_C:  9 10 True False
bignodes_c[cvec_self_idx] True
MUST USE HOUSEHOLDER 

In [24]:
b1 = r[DAG_algorithm]
b2 = r[scisparse_algorithm]

In [25]:
Meye1 = b1.Meye[2]
Meye2 = b2.Meye[2]

diag = []
offdiag = []
diag1 = []
offdiag1 = []
diag2 = []
offdiag2 = []
offdiag2x = []
for (k1, k2), e1 in Meye1.items():
    e2 = Meye2[k1, k2]
    e_diff = e1 - e2
    if k1 == k2:
        diag.append(e_diff)
        diag1.append(e1)
        diag2.append(e2)
    else:
        offdiag.append(e_diff)
        offdiag1.append(e1)
        offdiag2.append(e2)
        offdiag2x.append(b2.em0_ll[k1, k2])
diag = np.concatenate(diag)
offdiag = np.concatenate(offdiag)
diag1 = np.concatenate(diag1)
offdiag1 = np.concatenate(offdiag1)
diag2 = np.concatenate(diag2)
offdiag2 = np.concatenate(offdiag2)
offdiag2x = np.concatenate(offdiag2x)

In [26]:
axB = mplfigB(Nrows=4)
Nbins = 30

_ = axB.ax0.hist(b1.all0, normed = 1, bins = Nbins)
_ = axB.ax0.hist(b1.all1, normed = 1, alpha = .5, bins = Nbins)
axB.ax0.set_yscale('log')
print(np.min(b1.all0), np.max(b1.all0))
print(np.min(b1.all1), np.max(b1.all1))
axB.ax0.set_xlim(0, 30)

_ = axB.ax1.hist(b2.all0, normed = 1, bins = Nbins)
_ = axB.ax1.hist(b2.all1, normed = 1, alpha = .5, bins = Nbins)
print(np.min(b2.all0), np.max(b2.all0))
print(np.min(b2.all1), np.max(b2.all1))
axB.ax1.set_yscale('log')
axB.ax1.set_xlim(0, 30)

_ = axB.ax2.hist(-np.log10(np.maximum(abs(offdiag), 1e-30)), normed = 1, bins = Nbins)
_ = axB.ax2.hist(-np.log10(np.maximum(abs(diag), 1e-30)), normed = 1, alpha = .5, bins = Nbins)
#axB.ax2.set_yscale('log')
axB.ax2.set_xlim(0, 30)

_ = axB.ax3.hist(np.log10(r['condition']))

axB.save('comparison')

5.9336466418 30.0
6.52638913952 30.0
0.642254890468 30.0
2.47699795108 30.0
figure: plots/comparison.png
[[file:plots/comparison.png]]


In [27]:
r['condition']

array([  3.60227746e+09,   2.81705498e+09,   8.32756668e+09,
         5.38843565e+09,   4.98597902e+09,   6.62099998e+09,
         6.30020827e+09,   6.56117264e+09,   7.66802544e+09,
         7.20822440e+09,   7.34296421e+09,   6.50027264e+09,
         3.92982303e+09,   4.43891250e+09,   9.68726750e+09,
         7.78612681e+09,   5.87094820e+09,   6.49437023e+09,
         5.18773505e+09,   7.04627949e+09,   8.28830150e+09,
         7.20898637e+09,   5.94527248e+09,   6.96450365e+09,
         8.78689513e+09,   5.86026892e+09,   6.91921156e+09,
         5.03538464e+09,   6.41109643e+09,   7.56660913e+09,
         8.19085684e+09,   8.26666020e+09,   7.47408319e+09,
         6.36734628e+09,   7.80786552e+09,   6.15910294e+09,
         8.17461555e+09,   9.16563196e+09,   6.69267569e+09,
         4.85719292e+09,   7.61234669e+09,   3.63130992e+09,
         3.79903581e+09,   4.02712722e+09,   3.94355387e+09,
         4.41795589e+09,   2.36279620e+09,   6.89633852e+09,
         2.70215424e+09,

In [28]:
idx_max = np.argmax(np.log10(abs(offdiag2)))
print(-np.log10(abs(offdiag2[idx_max])))
print(offdiag2x[idx_max])


0.642254890468
0.642254890468


/home/mcculler/.local/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in log10
  if __name__ == '__main__':


In [29]:
idx_max = np.argmax(np.log10(abs(offdiag1)))
print(-np.log10(abs(offdiag1[idx_max])))

5.9336466418


/home/mcculler/.local/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in log10
  if __name__ == '__main__':
